# XGBoost for predicting rent price

In our [previous model](https://github.com/freirerene/apartamentos_python/blob/main/random_forest.ipynb) we applied a random forest regressor to the data we collect on the rent price of apartaments in São Paulo.

In this notebook we're going to apply [xgboost](https://xgboost.readthedocs.io/en/latest/), which is an open source implementation of gradient boosting. Gradient boosting is an improvement upon random forest: while random forests build a bunch of decision trees from randomly selected subsets of features and samples to build a tree, and in the end avarage them out, gradient boost builds each tree at a time, and each tree improves upon an earlier tree.

We chose to build this model using AWS because its faster this way -- we can do hyperparameter tuning in a couple of hours, as opposed to whole weeks.

Now, working on AWS is a bit different than simply running a jupyter notebook locally. First of all, we have to initiate the session within sagemaker, create a IAM (identify and access management) role and create an s3 (simple storage service) storage -- s3 is where sagemaker stores trainig/testing variables, as well as trainig jobs, hyperparameter tuning jobs, etc.

In [1]:
import sagemaker

# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

prefix = 'sagemaker/apartaments-xgboost'
region = boto3.Session().region_name
role = get_execution_role()
sess = sagemaker.Session()
bucket = sess.default_bucket()
sm_client = boto3.Session().client('sagemaker')

In order to select the best parameters, its easier to simply apply RFECV on the standard python implementation of xgboost. So we install the xgboost:

In [2]:
!pip install xgboost

     |████████████████████████████████| 157.5 MB 26 kB/s s eta 0:00:01


Now we go on to importing the necessary libraries.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
import pickle

from sagemaker.inputs import TrainingInput
from sagemaker.serializers import CSVSerializer
from sagemaker.tuner import IntegerParameter
from sagemaker.tuner import ContinuousParameter
from sagemaker.tuner import HyperparameterTuner

import xgboost
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.feature_selection import RFECV

We read the data and exclude the outliers, as well as restrict the area to 200 squared meters -- as we did in our other models.

In [4]:
df = pd.read_csv('https://github.com/freirerene/apartamentos_python/raw/main/apartamentos_clean.csv')

In [5]:
con = ['aluguel', 'condominio', 'area']
df = df[(np.abs(stats.zscore(df[con])) < 3).all(axis=1)]
df_b = pd.get_dummies(df, columns=['bairro'])
smaller_b = df_b[df_b['area'] <= 200]

## Functions

Here we define two functions we'll use below: a `print_mae`, whose only purpose is to print the MAE of the test and training set and the difference between them; the `predict` function, which separates the dataset we want to predict into chunks and applies our predictor to it.

In [6]:
def predict(data, model, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, model.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

In [7]:
def print_mae(y_test, y_train, y_p_test, y_p_train):
    mae_train = mean_absolute_error(y_train, y_p_train)
    mae_test = mean_absolute_error(y_test, y_p_test)
    spread = np.abs(mae_train - mae_test)
    print("MAE train: {:0.2f}".format(mae_train))
    print("-"*100)
    print("MAE test: {:0.2f}".format(mae_test))
    print("-"*100)
    print("Spread: {:0.2f}".format(spread))

## Feature selection

To select the features we'll use we will simply run an `XGBRegressor` with default parameters on a RFECV. What RFECV does is quite simple: it tests a combination of features and recursively excludes the combinations that don't produce good results. And it does this using cross validation.

First we define our features `x` and our target variable `y` below.

In [8]:
x = smaller_b.drop(['aluguel', 'anunciante', 'condominio'], axis=1)
y = smaller_b['aluguel'] + smaller_b['condominio']

In [ ]:
import xgboost
xgb_f = xgboost.XGBRegressor()

selector = RFECV(xgb_f, scoring='neg_mean_absolute_error', cv=3)
selector.fit(x, y)

We'll save the results on a `.data` file, so we can use it later without having to run this `selector.fit` again.

In [ ]:
optimized_columns = x.columns[selector.support_]

with open('optimized_columns.data', 'wb') as filehandle:
    pickle.dump(optimized_columns, filehandle)

In the two cells below we load the data we saved earlier and restrict the features to the ones we selected.

In [9]:
with open('optimized_columns.data', 'rb') as filehandle:
    optimized_columns = pickle.load(filehandle)

In [10]:
x = x[optimized_columns]

## Create train set and test set

To train our model we create two datasets: `train` and `test`, and we split the df using the usual sklearn's `train_test_split`. AWS's xgboost requires that the target variable is in the first column and that there's no label row, so we concat the `y`s and `X`s the way we did and we saved the data into `csv` without the header

In [11]:
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state = 1)

In [12]:
train = pd.concat([y_train, X_train], axis=1)
test = pd.concat([y_test, X_test], axis=1)

In [13]:
train.to_csv('train.csv', header=False, index=False)
test.to_csv('validation.csv', header=False, index=False)

Below we load the data into an s3 instance and load it back as an s3 input variable.

In [14]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')

In [15]:
s3_input_train = TrainingInput(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = TrainingInput(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')

## The model

Now we initiate xgboost sagemaker's built-in algorithm and create an estimator -- in the estimator we can use a slightly better instance (our base plan is `ml.t2.medium`, which is the most basic, but in some jobs AWS allows us to jump to better instances and then jump basic to our basic instance. This is good for price optimization).

In [16]:
container = sagemaker.image_uris.retrieve('xgboost', region, version='latest')

In [18]:
xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m5.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)


xgb.set_hyperparameters(objective='reg:linear', num_round=100)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

2021-03-02 22:31:05 Starting - Starting the training job...
2021-03-02 22:31:28 Starting - Launching requested ML instancesProfilerReport-1614724265: InProgress
......
2021-03-02 22:32:29 Starting - Preparing the instances for training...
2021-03-02 22:33:01 Downloading - Downloading input data
2021-03-02 22:33:01 Training - Downloading the training image..Arguments: train
[2021-03-02:22:33:15:INFO] Running standalone xgboost training.
[2021-03-02:22:33:15:INFO] File size need to be processed in the node: 4.85mb. Available memory size in the node: 8078.93mb
[2021-03-02:22:33:15:INFO] Determined delimiter of CSV input is ','
[22:33:15] S3DistributionType set as FullyReplicated
[22:33:15] 13514x133 matrix with 1797362 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2021-03-02:22:33:15:INFO] Determined delimiter of CSV input is ','
[22:33:15] S3DistributionType set as FullyReplicated
[22:33:15] 4505x133 matrix with 599165 entries loaded from /opt/ml/inp


2021-03-02 22:33:30 Uploading - Uploading generated training model
2021-03-02 22:33:30 Completed - Training job completed
Training seconds: 40
Billable seconds: 40


Now we deploy our model (this crates an endpoint that can be used in other notebooks, and so on) and test it.

In [19]:
xgb_predictor = xgb.deploy(
    initial_instance_count = 1, 
    instance_type = 'ml.t2.medium',
    serializer=CSVSerializer())

---------------!

In [20]:
predictions_train = predict(train.to_numpy()[:,1:], xgb_predictor)
predictions_test = predict(test.to_numpy()[:,1:], xgb_predictor)
print_mae(y_test, y_train, predictions_test, predictions_train)

MAE train: 622.33
----------------------------------------------------------------------------------------------------
MAE test: 724.16
----------------------------------------------------------------------------------------------------
Spread: 101.83


We see that these results are already better than the best results using random forests. Now let's see if we can improve it doing some hyperparameter tuning.

# Hyperparameter tuning

To perform the tuning we use AWS' `HyperparameterTuner`. We simply define the hyperparameter's range in a dictionary. With `tuner_log` we define the job then fit it to our data.

We can see the best result in our sagemaker's dashboard, in the `Training -> Hyperparameter tuning jobs` section.

In [ ]:
hyperparameter_ranges = {
    'alpha': ContinuousParameter(0, 1000, scaling_type="Auto"),
    'subsample': ContinuousParameter(0.5,1,scaling_type='Logarithmic'),
    'num_round': IntegerParameter(1, 4000, scaling_type="Auto"),
    'min_child_weight': ContinuousParameter(0, 120, scaling_type="Auto")
}

tuner = HyperparameterTuner(
    xgb,
    objective_metric_name = 'validation:mae',
    objective_type ='Minimize',
    hyperparameter_ranges = hyperparameter_ranges,
    max_jobs=150,
    max_parallel_jobs=4,
    strategy='Bayesian'
)

tuner.fit({'train': s3_input_train, 'validation': s3_input_validation}, include_cls_metadata=False)

To get the results from the tuning we do the following:

In [21]:
tuning_job_result = sm_client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName = 'xgboost-210227-0429')

One can also get the name of the (last) tuning job with the command `tuner.latest_tuning_job.job_name`.

And, finally, to see the best parameters from the tuning job we simply have to find the right keys in the dictionary `tuning_job_result`:

In [22]:
tuning_job_result['BestTrainingJob']['TunedHyperParameters']

{'alpha': '0.0',
 'min_child_weight': '0.14731765645826228',
 'num_round': '807',
 'subsample': '0.949738441523912'}

In [23]:
xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m5.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)


xgb.set_hyperparameters(objective='reg:linear',
                        alpha=0.0,
                        min_child_weight=0.14731765645826228,
                        subsample=0.949738441523912,
                        num_round=807)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

2021-03-02 22:44:20 Starting - Starting the training job...
2021-03-02 22:44:43 Starting - Launching requested ML instancesProfilerReport-1614725060: InProgress
......
2021-03-02 22:45:44 Starting - Preparing the instances for training...
2021-03-02 22:46:15 Downloading - Downloading input data
2021-03-02 22:46:15 Training - Downloading the training image..Arguments: train
[2021-03-02:22:46:29:INFO] Running standalone xgboost training.
[2021-03-02:22:46:29:INFO] File size need to be processed in the node: 4.85mb. Available memory size in the node: 8084.64mb
[2021-03-02:22:46:29:INFO] Determined delimiter of CSV input is ','
[22:46:29] S3DistributionType set as FullyReplicated
[22:46:29] 13514x133 matrix with 1797362 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2021-03-02:22:46:29:INFO] Determined delimiter of CSV input is ','
[22:46:29] S3DistributionType set as FullyReplicated
[22:46:29] 4505x133 matrix with 599165 entries loaded from /opt/ml/inp


2021-03-02 22:46:45 Training - Training image download completed. Training in progress.[22:46:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 94 extra nodes, 0 pruned nodes, max_depth=6
[301]#011train-rmse:668.046#011validation-rmse:1037.39
[22:46:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 94 extra nodes, 0 pruned nodes, max_depth=6
[302]#011train-rmse:667.243#011validation-rmse:1036.97
[22:46:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 96 extra nodes, 0 pruned nodes, max_depth=6
[303]#011train-rmse:666.245#011validation-rmse:1036.76
[22:46:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=6
[304]#011train-rmse:665.947#011validation-rmse:1036.73
[22:46:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 74 extra nodes, 0 pruned nodes, max_depth=6
[305]#011train-rmse:665.478#011validation-rmse:1036.63
[22:46:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra 

[22:46:49] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 90 extra nodes, 0 pruned nodes, max_depth=6
[638]#011train-rmse:570.509#011validation-rmse:1059.25
[22:46:49] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 106 extra nodes, 0 pruned nodes, max_depth=6
[639]#011train-rmse:570.172#011validation-rmse:1059.37
[22:46:49] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 102 extra nodes, 0 pruned nodes, max_depth=6
[640]#011train-rmse:569.977#011validation-rmse:1059.78
[22:46:49] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 100 extra nodes, 0 pruned nodes, max_depth=6
[641]#011train-rmse:569.822#011validation-rmse:1059.82
[22:46:49] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 80 extra nodes, 0 pruned nodes, max_depth=6
[642]#011train-rmse:569.676#011validation-rmse:1060.63
[22:46:49] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 88 extra nodes, 0 pruned nodes, max_depth=6
[643]#011train-rmse:569.552#011validation-rmse:106


2021-03-02 22:47:05 Uploading - Uploading generated training model
2021-03-02 22:47:05 Completed - Training job completed
Training seconds: 61
Billable seconds: 61


In [24]:
xgb_predictor_opt = xgb.deploy(
    initial_instance_count = 1, 
    instance_type = 'ml.t2.medium',
    serializer=CSVSerializer())

-----------------!

In [25]:
predictions_train = predict(train.to_numpy()[:,1:], xgb_predictor_opt)
predictions_test = predict(test.to_numpy()[:,1:], xgb_predictor_opt)
print_mae(y_test, y_train, predictions_test, predictions_train)

MAE train: 355.60
----------------------------------------------------------------------------------------------------
MAE test: 648.89
----------------------------------------------------------------------------------------------------
Spread: 293.29


In [26]:
xgb_predictor_opt.delete_endpoint()

## Control overfit

There are two straightforward ways to control overfitting adjusting the hyperparameters: first, controlling the model complexity by adjusting the `max_depth` and the `gamma` (aka `min_split_loss`); second, by making the model more robust to noises, by adjusting `colsample_bytree`.

There are other parameters that control model complexity and its behaviour with noises, and some of them were already optimized in the tuning job earlier. This actually gives a way better result compared to just tuning every single parameter possible.

### Model complexity

After some experimentation we saw that a good compromise in the bias-variance tradeoff is to set `max_depth=5` and `gamma=5`.

In [27]:
xgb2 = sagemaker.estimator.Estimator(container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m5.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)


xgb2.set_hyperparameters(objective='reg:linear',
                        gamma=5,
                        max_depth=5,
                        alpha=0.0,
                        min_child_weight=0.14731765645826228,
                        subsample=0.949738441523912,
                        num_round=807)

xgb2.fit({'train': s3_input_train, 'validation': s3_input_validation})

2021-03-02 22:56:54 Starting - Starting the training job...
2021-03-02 22:57:18 Starting - Launching requested ML instancesProfilerReport-1614725814: InProgress
......
2021-03-02 22:58:19 Starting - Preparing the instances for training...
2021-03-02 22:58:52 Downloading - Downloading input data
2021-03-02 22:58:52 Training - Downloading the training image..Arguments: train
[2021-03-02:22:59:07:INFO] Running standalone xgboost training.
[2021-03-02:22:59:07:INFO] File size need to be processed in the node: 4.85mb. Available memory size in the node: 8078.17mb
[2021-03-02:22:59:07:INFO] Determined delimiter of CSV input is ','
[22:59:07] S3DistributionType set as FullyReplicated
[22:59:07] 13514x133 matrix with 1797362 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2021-03-02:22:59:07:INFO] Determined delimiter of CSV input is ','
[22:59:07] S3DistributionType set as FullyReplicated
[22:59:07] 4505x133 matrix with 599165 entries loaded from /opt/ml/inp


2021-03-02 22:59:20 Training - Training image download completed. Training in progress.[22:59:12] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=5
[171]#011train-rmse:844.211#011validation-rmse:1055.53
[22:59:12] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 0 pruned nodes, max_depth=5
[172]#011train-rmse:843.116#011validation-rmse:1054.15
[22:59:12] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 0 pruned nodes, max_depth=5
[173]#011train-rmse:840.793#011validation-rmse:1053.44
[22:59:12] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 0 pruned nodes, max_depth=5
[174]#011train-rmse:840.462#011validation-rmse:1053.05
[22:59:12] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[175]#011train-rmse:840.068#011validation-rmse:1052.91
[22:59:12] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra 

[22:59:22] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=5
[560]#011train-rmse:655.519#011validation-rmse:1048.31
[22:59:22] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[561]#011train-rmse:655.122#011validation-rmse:1048.15
[22:59:22] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 0 pruned nodes, max_depth=5
[562]#011train-rmse:654.857#011validation-rmse:1048.17
[22:59:22] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=5
[563]#011train-rmse:654.829#011validation-rmse:1048.3
[22:59:22] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[564]#011train-rmse:654.635#011validation-rmse:1047.98
[22:59:22] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=5
[565]#011train-rmse:654.329#011validation-rmse:1048.57


2021-03-02 22:59:41 Uploading - Uploading generated training model
2021-03-02 22:59:41 Completed - Training job completed
Training seconds: 63
Billable seconds: 63


In [28]:
xgb_predictor2 = xgb2.deploy(
    initial_instance_count = 1, 
    instance_type = 'ml.t2.medium',
    serializer=CSVSerializer())

---------------!

In [29]:
predictions_train = predict(train.to_numpy()[:,1:], xgb_predictor2)
predictions_test = predict(test.to_numpy()[:,1:], xgb_predictor2)
print_mae(y_test, y_train, predictions_test, predictions_train)

MAE train: 413.48
----------------------------------------------------------------------------------------------------
MAE test: 662.50
----------------------------------------------------------------------------------------------------
Spread: 249.02


In [30]:
xgb_predictor2.delete_endpoint()

### Noise robustness

Like in the complexity control, we saw from experimentation that the best `colsample_bytree` is `1.0`:

In [31]:
xgb3 = sagemaker.estimator.Estimator(container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m5.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)


xgb3.set_hyperparameters(objective='reg:linear',
                        gamma=5,
                        max_depth=5,
                        alpha=0.0,
                        min_child_weight=0.14731765645826228,
                        subsample=0.949738441523912,
                        colsample_bytree=1.0,
                        num_round=807)

xgb3.fit({'train': s3_input_train, 'validation': s3_input_validation})

2021-03-02 23:09:14 Starting - Starting the training job...
2021-03-02 23:09:37 Starting - Launching requested ML instancesProfilerReport-1614726553: InProgress
......
2021-03-02 23:10:37 Starting - Preparing the instances for training......
2021-03-02 23:11:38 Downloading - Downloading input data
2021-03-02 23:11:38 Training - Downloading the training image.Arguments: train
[2021-03-02:23:11:46:INFO] Running standalone xgboost training.
[2021-03-02:23:11:46:INFO] File size need to be processed in the node: 4.85mb. Available memory size in the node: 8087.02mb
[2021-03-02:23:11:46:INFO] Determined delimiter of CSV input is ','
[23:11:46] S3DistributionType set as FullyReplicated
[23:11:47] 13514x133 matrix with 1797362 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2021-03-02:23:11:47:INFO] Determined delimiter of CSV input is ','
[23:11:47] S3DistributionType set as FullyReplicated
[23:11:47] 4505x133 matrix with 599165 entries loaded from /opt/ml/i

[23:11:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=5
[103]#011train-rmse:914.692#011validation-rmse:1085.36
[23:11:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 0 pruned nodes, max_depth=5
[104]#011train-rmse:914.268#011validation-rmse:1085.08
[23:11:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=5
[105]#011train-rmse:913.874#011validation-rmse:1084.83
[23:11:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[106]#011train-rmse:911.27#011validation-rmse:1085.74
[23:11:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[107]#011train-rmse:910.187#011validation-rmse:1084.94
[23:11:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 0 pruned nodes, max_depth=5
[108]#011train-rmse:909.16#011validation-rmse:1084.51



2021-03-02 23:12:12 Uploading - Uploading generated training model[23:12:00] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[500]#011train-rmse:672.668#011validation-rmse:1046.76
[23:12:00] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=5
[501]#011train-rmse:672.226#011validation-rmse:1047.13
[23:12:00] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[502]#011train-rmse:672.1#011validation-rmse:1046.88
[23:12:00] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 0 pruned nodes, max_depth=5
[503]#011train-rmse:671.72#011validation-rmse:1046.78
[23:12:00] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=5
[504]#011train-rmse:671.541#011validation-rmse:1046.94
[23:12:00] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, m


2021-03-02 23:12:39 Completed - Training job completed
Training seconds: 64
Billable seconds: 64


In [32]:
xgb_predictor3 = xgb3.deploy(
    initial_instance_count = 1, 
    instance_type = 'ml.t2.medium',
    serializer=CSVSerializer())

-----------------!

In [30]:
predictions_train = predict(train.to_numpy()[:,1:], xgb_predictor3)
predictions_test = predict(test.to_numpy()[:,1:], xgb_predictor3)
print_mae(y_test, y_train, predictions_test, predictions_train)

MAE train: 413.48
----------------------------------------------------------------------------------------------------
MAE test: 662.50
----------------------------------------------------------------------------------------------------
Spread: 249.02


# Conclusion

If we poke around the model we could improve it further, but the results are quite good -- we were able to decrease the MAE by `20%`, comparing our best random forest model with our best xgboost model, even though the spread is a tad higher than ideal.

We end the project with two endpoints: `xgb_predictor` and `xgb_predictor3`.  The first represents the model without any optimized parameter; the second represents the final model.